In [3]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk import sent_tokenize
from gensim.utils import simple_preprocess
nltk.download('punkt')

from sklearn.preprocessing import StandardScaler
import pickle
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
def onehot(ft,dataset):
    label = [x for x in dataset[ft].value_counts().head(10).index]
    for i in label:
        dataset[i] = np.where(dataset[ft] ==i,1,0)

def test(file):

    data = pd.read_csv(file,usecols = ['Summary','Issue Type','Status','Assignee','Priority','Created','Major blocks','Sub-blocks'])
    i = data[(data['Issue Type'] != 'Bug')].index
    data = data.drop(i,axis=0)

    df = pd.DataFrame(columns = ['Summary','Majorblocks','Sub-blocks','Assignee'])
    df.Summary = data['Summary']
    df.Assignee = data['Assignee']
    df.Majorblocks = data['Major blocks']


    label_encoder = preprocessing.LabelEncoder()
    data['Status'] = label_encoder.fit_transform(data['Status'])
    data['Priority'] = label_encoder.fit_transform(data['Priority'])
    data['Major blocks'] = label_encoder.fit_transform(data['Major blocks'])

    onehot('Assignee',data)

    data['Date'] = pd.to_datetime(data['Created']).dt.date
    data['Creatd_Time'] = pd.to_datetime(data['Created']).dt.time

    data["created_day"] = pd.to_datetime(data.Date, format="%Y-%m-%d").dt.day
    data["created_month"] = pd.to_datetime(data.Date, format="%Y-%m-%d").dt.month

    # ind=data['Date'].index
    # ind = list(ind)
    # k=0
    # index = []
    # for i in data['Date'].values:
    #     if dt.strptime(str(i), "%Y-%m-%d")  >  dt.strptime("2022-09-27", "%Y-%m-%d"):
    #         # if (dt.strptime(str(j), "%H:%M:%S") > dt.strptime("20:41:00", "%H:%M:%S")):
    #         index.append(ind[k])
    #         k=k+1
    # print(index)

    df['Date'] = data['Date']
    df['Time'] = data['Creatd_Time']
    

    data = data.drop(['Date','Created','Issue Type','Assignee','Creatd_Time'],axis=1)

   

    data['Summary'] = data['Summary'].apply(lambda x : re.sub('[^a-zA-Z0-9]',' ',x))
    corpus1 = []
    index1 = list(data.index)
    i=0
    lemmatizer=WordNetLemmatizer()
    for j in range(0,len(data.Summary)):
        title = data.Summary[index1[i]]
        title = title.lower()
        title = title.split()
        title = [lemmatizer.lemmatize(word) for word in title if not word in stopwords.words('english')]
        title = ' '.join(title)
        corpus1.append(title)
        i=i+1

    from sklearn.feature_extraction.text import TfidfVectorizer
    tv = TfidfVectorizer(max_features=2500)
    X = tv.fit_transform(corpus1).toarray()
    data.Summary = X
    
    

    sc = StandardScaler()
    x = data
    x = sc.fit_transform(x)


    model1 = open("RFS_SUB.pkl",'rb')
    #model2 = open("RFH.pkl",'rb')

    model1_RFS = pickle.load(model1)

    pred1 = model1_RFS.predict(x)

    index = data.index
    j=0
    for i in pred1:
        df['Sub-blocks'][index[j]] = i
        j=j+1
    
    df.to_csv('res_sub.csv')
    print(df.head(20))


    

    #print(pred1)
    #print(pred2)

In [9]:
test("/content/sub_test.csv")

                                              Summary      Majorblocks  \
0   LSU FV: Store cross cache misaligned with 2nd ...  Load Store Unit   
1        [TGV] Correlation: Look into vfmerge latency        Interrupt   
2   [TGV]:: vsetvli instruction not retaining prev...  Execution Unit    
3   Design considering the wrong cause code for Ma...        Interrupt   
4   Assertion in LST pipe of LSU On L2 testing of ...  Load Store Unit   
5   [TGV] LSU requesting to to claim an LTB in Bal...  Load Store Unit   
6                       [TGV] vmv1r.v WRData Mismatch  Execution Unit    
7   [TGV] Assert: LSTO Store Data Issued must matc...  Load Store Unit   
8   [TGV] Assertion Failure : VexPipe : credit ove...  Execution Unit    
9                      [TGV] vsetResolveStall_ID Hang  Execution Unit    
10  [TGV] LSU sends wrong load data to LTB (differ...  Load Store Unit   
11  [TGV] Recently added assertion in MallardTileV...  Execution Unit    
12      [TGV] LSU: lsta_srcReg unstabl

In [4]:
from datetime import datetime as dt